# Обучение и тестирование

<div style="text-align: right"> М. М. Шамшиев  </div> 
<div style="text-align: right"> 06.05.2018 </div> 

In [1]:
import numpy as np
import glob
import skimage.io
from sklearn.model_selection import train_test_split

from ConvNetwork import ConvNetAutoEncoder
from ConvNetwork import fit_net
from ConvNetwork import check_accuracy
from ConvNetwork import loader_from_numpy
from ConvNetwork import get_predictions

## Загрузка и подготовка данных

Предполагается, что папки, содержащие изображения, находятся в текущей директории и имеют названия "crocodiles" и "clocks" (если это не так, укажите верный путь в ячейке ниже).

In [2]:
crocs = np.array([skimage.io.imread(file) for file in glob.glob("crocodiles/*.png")])
crocs = np.transpose(crocs, (0, 3, 1, 2))

clocks = np.array([skimage.io.imread(file) for file in glob.glob("clocks/*.png")])
clocks = np.transpose(clocks, (0, 3, 1, 2))

In [3]:
X = np.concatenate((crocs, clocks))
y = np.concatenate((np.zeros(len(crocs), dtype=np.int), np.ones(len(clocks), dtype=np.int)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [4]:
trainloader = loader_from_numpy(X_train, y_train)
testloader = loader_from_numpy(X_test, y_test, shuffle=False)

## Обучение модели

In [5]:
net = ConvNetAutoEncoder(input_size=(3, 32, 32), conv_layers_num=1, conv_out_channels=25, conv_kernel_size=4, 
                         conv_stride=2, pool_kernel_size=2, pool_stride=1)

In [6]:
fit_net(net, trainloader, num_epoch=15, verbose=True)

[epoch 5] loss: 0.397
[epoch 10] loss: 0.215
[epoch 15] loss: 0.149


0.14910368936794943

## Предсказание

С помощью функции get_predictions() получим скоры классов для объектов тестовой выборки:

In [7]:
scores = get_predictions(net, testloader)

Непосредственно предсказания модели (метки классов) можно получить следующим образом:

In [8]:
predictions = np.argmax(scores, axis=1)

Подсчитаем точность:

In [9]:
accuracy = (predictions == y_test).sum() / len(y_test)
print(accuracy)

0.8833333333333333


Получить точность предсказания сразу можно было бы, воспользовавшись функцией check_accuracy():

In [10]:
check_accuracy(net, testloader, verbose=True)

Accuracy of the network: 88.33 %


88.33333333333333